In [ ]:
# importing all the modules 

import sys
import math
import laspy
import pyproj

import numpy as np
import pandas as pd
import geopandas as gpd

from datetime import datetime
from pycrown import PyCrown
from shapely.geometry import Polygon

# Import my own functions
from my_functions.wcc import *
from my_functions.jucker import *

In [ ]:
# Define the input folder where the files are stored
input_folder = 'input'

# Paths to the input files including DSM, DTM, CHM, and LiDAR point cloud,
# already clipped to the study site extent.
F_CHM = f'{input_folder}/clipped_CHM.tif'
F_DTM = f'{input_folder}/clipped_DTM.tif'
F_DSM = f'{input_folder}/clipped_DSM.tif'
F_LAS = f'{input_folder}/clipped_pointcloud.las'

In [ ]:
# Initialize the PyCrown class with the provided input files
# F_CHM: Path to the Canopy Height Model (CHM) file in .tif format
# F_DTM: Path to the Digital Terrain Model (DTM) file in .tif format
# F_DSM: Path to the Digital Surface Model (DSM) file in .tif format
# F_LAS: Path to the LiDAR point cloud file in .las format
# outpath: The directory where the output results will be saved
PC = PyCrown(F_CHM, F_DTM, F_DSM, F_LAS, outpath='result')

In [ ]:
# Smoothing of CHM while preserving fine details in the CHM
# The method filter_chm applies a smoothing filter to the Canopy Height Model (CHM).
# Parameters:
# 1: The window size in pixels for smoothing. A value of 1 means no smoothing is applied,
#    as each pixel is considered individually without averaging neighboring values.

# ws_in_pixels=True: Specifies that the window size is given in pixels, not in some other unit.
# circular=False: Indicates that a square window is used for filtering instead of a circular one.

PC.filter_chm(1, ws_in_pixels=True, circular=False)

In [ ]:
# Tree detection in the CHM using specified parameters
# The method tree_detection identifies tree tops in the Canopy Height Model (CHM).
# Parameters:
# PC.chm: The Canopy Height Model to be analyzed.
# ws=1: A window size of 1x1 pixel, meaning each pixel is considered independently as a potential tree top.
# hmin=1.3: The minimum height threshold for a pixel to be considered as a tree top. 
#   This has been adjusted from 16.0 to 1.3 meters to detect smaller trees.

PC.tree_detection(PC.chm, ws=1, hmin=1.3)

In [ ]:
# Delineation of tree crowns using the specified algorithm and thresholds
# The method crown_delineation is used to define the boundaries of tree crowns in the CHM.

# Define parameters for crown delineation
algorithm = 'dalponteCIRC_numba'  # Algorithm used for crown delineation
th_tree = 1.3  # Minimum height threshold for considering a tree crown in meters
th_seed = 0.5  # Seed threshold factor
th_crown = 0.05  # Crown threshold factor
max_crown = 4.0  # Maximum allowable crown radius in meters

# Delineation of tree crowns using the specified algorithm and thresholds
PC.crown_delineation(algorithm=algorithm,
                     th_tree=th_tree,
                     th_seed=th_seed,
                     th_crown=th_crown,
                     max_crown=max_crown)

In [ ]:
# Correcting tree tops after initial detection
# The correct_tree_tops() function refines the positions of detected tree tops.
# This process includes:
# - Removing false positives: tree tops that were incorrectly identified due to noise or low thresholds.
# - Adjusting the position of tree tops to more accurate locations within the tree crowns.
# - Merging closely positioned tree tops that likely represent the same tree.

# This correction step ensures that the final tree top positions are more accurate, 
# leading to better results in subsequent analyses.
PC.correct_tree_tops()

In [ ]:
# Calculate tree height and elevation for the detected tree tops
PC.get_tree_height_elevation(loc='top')

# Calculate tree height and elevation for the corrected tree tops
PC.get_tree_height_elevation(loc='top_cor')

In [ ]:
# Screen out small trees from the detected tree tops
# hmin=0.2: Minimum height threshold for a tree is 0.2 meters.
PC.screen_small_trees(hmin=0.2, loc='top') 

In [ ]:
# Convert detected tree crowns to polygon shapes using a raster-based approach
PC.crowns_to_polys_raster()

# Convert detected tree crowns to smoothed polygon shapes
# and optionally store the corresponding LiDAR points for each crown
PC.crowns_to_polys_smooth(store_las=True)

In [ ]:
# Perform quality control on the detected and processed tree crowns
PC.quality_control()

In [ ]:
print(f"Number of trees detected: {len(PC.trees)}")

In [ ]:
export_data = False  # or False, depending on your condition

if export_data:
    # Export the Canopy Height Model (CHM) as a raster file
    PC.export_raster(PC.chm, PC.outpath / 'chm.tif', 'CHM')

    # Export the locations of the initially detected tree tops
    PC.export_tree_locations(loc='top')

    # Export the locations of the corrected tree tops
    PC.export_tree_locations(loc='top_cor')

    # Export the tree crowns as raster-based polygons
    PC.export_tree_crowns(crowntype='crown_poly_raster')

    # Export the tree crowns as smoothed polygons
    PC.export_tree_crowns(crowntype='crown_poly_smooth')
else:
    print("Export skipped")

In [ ]:
# AIM: Create a dataset with the identified trees, their heights and location (latitude and longitude)

# Convert your DataFrame to a GeoDataFrame using the 'top_cor' column for geometry
# 'top_cor' is assumed to be a column containing geometries (e.g., points representing tree tops)
trees_gdf = gpd.GeoDataFrame(PC.trees, geometry='top_cor')

# Setting the Coordinate Reference System (CRS) for the GeoDataFrame
# EPSG:27700 corresponds to OSGB36 / British National Grid, which is commonly used in the UK
epsg_code = "EPSG:27700"
trees_gdf = trees_gdf.set_crs(epsg_code)

# Function to calculate the diameter of a tree crown polygon
def polygon_diameter(polygon):
    if isinstance(polygon, Polygon):
        # Extract the bounding box of the polygon (minx, miny, maxx, maxy)
        minx, miny, maxx, maxy = polygon.bounds
        # Calculate the diameter as the maximum of the width or height of the bounding box
        diameter = max(maxx - minx, maxy - miny)
        return diameter
    else:
        return None

# Calculate the diameter for each tree crown polygon
# 'crown_poly_raster' is assumed to be a column containing polygon geometries for tree crowns
trees_gdf['diameter'] = trees_gdf['crown_poly_raster'].apply(polygon_diameter)


# Create a DataFrame with tree number, height, and calculated diameter
trees_gdf['tree_number'] = trees_gdf.index  # Assign a unique tree number based on the index
tree_database = trees_gdf[['tree_number', 'top_height', 'diameter']].copy()

# Add the geometry column back to the new DataFrame to make it a GeoDataFrame
tree_database = gpd.GeoDataFrame(tree_database, geometry=trees_gdf.geometry)

In [ ]:
# AIM: add a shapefile to identify trees within particular areas and then planting years#
# This adds to the DataFrame the type and the year to each tree

# Loading the shapefile of the planting areas
planting_types = f'{input_folder}/Planting Types.shp'
planting_areas_gdf = gpd.read_file(planting_types)

# Perform a spatial join to keep only trees within the planting areas
# 'how="inner"' keeps only the rows where the spatial join condition ('within') is true
trees_within_planting_areas = gpd.sjoin(tree_database, planting_areas_gdf, how='inner', op='within')

# Remove the 'index_right' and 'id' columns if present
# This cleans up the GeoDataFrame by removing unnecessary columns resulting from the join
columns_to_drop = ['index_right', 'id']
trees_within_planting_areas = trees_within_planting_areas.drop(columns=[col for col in columns_to_drop if col in trees_within_planting_areas.columns])

# planting years areas 
planting_years = f'{input_folder}/Planting Types.shp'
planting_years_gdf = gpd.read_file(planting_years)

# Perform a spatial join to add attributes from the planting years shapefile
# Again, 'how="inner"' ensures that only trees within the specified areas are kept
trees_within_both_areas = gpd.sjoin(trees_within_planting_areas, planting_years_gdf, how='inner', op='within')

# Clean up by removing any redundant columns from the second spatial join
columns_to_drop_second = ['index_right']  # Adjust based on actual column names resulting from the second join
trees_within_both_areas = trees_within_both_areas.drop(columns=[col for col in columns_to_drop_second if col in trees_within_both_areas.columns])

# Rename columns to provide meaningful names
trees_within_both_areas = trees_within_both_areas.rename(columns={'id': 'Year'})

In [ ]:
#len(trees_within_both_areas)
print(trees_within_both_areas.head(10))

In [ ]:
# Create a CSV file for further analysis
trees_within_both_areas.to_csv('trees_within_both_areas.csv', index=False)

In [ ]:
# Reading the CSV file into a DataFrame
df = trees_within_both_areas

In [ ]:
# AIM: add DBH to the data set of each tree
# Below section is based on Jucker et al. 2017 ("Allometric equations for integrating remote sensing imagery into forest monitoring programmes")

# Parameters for DBH calculation
exp_factor_d = np.exp(0.056**2 / 2)

# Calculate DBH (Diameter at Breast Height)
# Unit for DBH is cm 
df['DBH'] = 0.557 * (df['top_height'] * df['diameter'])**0.809 * exp_factor_d

In [ ]:
print(df)

# WCC FRAMEWORK # 

In [ ]:
# Initialize total aggregates
total_agb_all_species = 0
total_root_biomass_all_species = 0

# Iterate over all species in your dataset
for species in tree_stats['species_distribution'].keys():
    # Get the volume statistics for the species
    volume_stats = calculate_tariff_numbers_and_volume(tree_stats, species)

    # Calculate the biomass for the species
    biomass_stats = calculate_biomass(tree_stats, species, volume_stats)

    # Accumulate the total AGB and root biomass for all species
    total_agb_all_species += biomass_stats['total_AGB']
    total_root_biomass_all_species += biomass_stats['total_root_biomass']

# Compute the ratio of total root biomass to total AGB
root_to_agb_ratio = total_root_biomass_all_species / total_agb_all_species if total_agb_all_species != 0 else None

# Print the results
print(f"Total AGB for all species: {total_agb_all_species:.4f} oven-dry tonnes")
print(f"Total Root Biomass for all species: {total_root_biomass_all_species:.4f} oven-dry tonnes")
if root_to_agb_ratio is not None:
    print(f"Ratio of Total Root Biomass to Total AGB: {root_to_agb_ratio:.4f}")
else:
    print("Total AGB is zero, cannot compute ratio.")

In [ ]:
# Define species percentages and planting mix
species_percentages = {
    'oak': 0.40,  # 40% Oak
    'Scots pine': 0.30,  # 30% Scots Pine
    'European larch': 0.30  # 30% European Larch
}
planting_mix = 'Mixed Wood'
year = 2

# Assuming `df` is your DataFrame containing the tree data
tree_stats = tree_statistics(df, species_percentages, planting_mix, year)

print_tree_statistics(tree_stats)

# Calculate tree biomass and convert to carbon and CO2 for each species
total_aggregates = calculate_and_print_species_biomass_and_carbon(tree_stats, species_percentages)
# Print overall totals
print_overall_totals(total_biomass_all_species, total_carbon_all_species, total_co2_all_species, total_co2_trees_all_species, total_co2_saplings_all_species)

#  JUCKER ET AL. 2017 

### AGB- using crown diameter and height

In [ ]:
# reading the file 
df = pd.read_csv('trees_within_both_areas.csv')

In [ ]:
# Parameters for ABG 
exp_factor_agb = np.exp(0.204**2 / 2)

# As there is no option to classify the type of species of signular tree, it will be assumed that they could be equally (50% chance) either angiosperms and gymnosperms and then a waighted average will be applied  
# Calculate AGB for angiosperms and gymnosperms
# Unit for AGB is kg.
df['AGB_angiosperm'] = 0.016 * (df['top_height'] * df['diameter'])**2.013 * exp_factor_agb
df['AGB_gymnosperm'] = 0.109 * (df['top_height'] * df['diameter'])**1.790 * exp_factor_agb


In [ ]:
species_mix_proportions = {
    "Mixed Wood": (0.4, 0.6),  # 40% Angiosperms, 60% Gymnosperms
    "Native Bro": (1.0, 0.0),  # 100% Angiosperms
    "Native Shr": (1.0, 0.0),  # 100% Angiosperms
    "Native Tim": (1.0, 0.0),  # 100% Angiosperms
    "Shrub-rich": (1.0, 0.0)   # 100% Angiosperms
}

ratio_bgb_to_agb = 0.1667  # Given ratio of BGB to AGB

# Calculate the biomass summary
biomass_summary = calculate_biomass_summary(df, ratio_bgb_to_agb, species_mix_proportions)

In [ ]:
# Display the summary
print("Biomass and Carbon Content Summary by Planting Type and Year:")
print(biomass_summary)